In [8]:
import numpy as np
import logging
import sys
from matplotlib import pyplot as plt

%matplotlib inline
sys.path.append("../")

import torch
from torchvision.datasets import MNIST
from torchvision import transforms

from aef.models.autoencoder import LinearAutoencoder, ConvolutionalAutoencoder, DenseAutoencoder
from aef.trainer import AutoencoderTrainer
from aef.losses import nll, mse

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)


## Data

In [9]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist = MNIST('./data', download=True, transform=img_transform)

## Train autoencoder

In [10]:
ae = LinearAutoencoder(latent_dim=20)

In [11]:
trainer = AutoencoderTrainer(ae, output_filename="../figures/training/linear/linear_phase1")
trainer.train(
    dataset=mnist,
    loss_functions=[mse],
    loss_labels=["MSE"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5
)

11:04 aef.trainer          INFO    Epoch   1: train loss  0.38500 (MSE:  0.385)
11:04 aef.trainer          INFO               val. loss   0.23895 (MSE:  0.239)
11:04 aef.trainer          INFO    Epoch   2: train loss  0.21967 (MSE:  0.220)
11:04 aef.trainer          INFO               val. loss   0.20023 (MSE:  0.200)
11:04 aef.trainer          INFO    Epoch   3: train loss  0.18936 (MSE:  0.189)
11:04 aef.trainer          INFO               val. loss   0.17946 (MSE:  0.179)
11:05 aef.trainer          INFO    Epoch   4: train loss  0.17420 (MSE:  0.174)
11:05 aef.trainer          INFO               val. loss   0.16925 (MSE:  0.169)
11:05 aef.trainer          INFO    Epoch   5: train loss  0.16651 (MSE:  0.167)
11:05 aef.trainer          INFO               val. loss   0.16388 (MSE:  0.164)
11:05 aef.trainer          INFO    Epoch   6: train loss  0.16231 (MSE:  0.162)
11:05 aef.trainer          INFO               val. loss   0.16077 (MSE:  0.161)
11:05 aef.trainer          INFO    Epoch

(array([0.38499626, 0.21967436, 0.18936219, 0.17420215, 0.16651253,
        0.16230597, 0.15986796, 0.15841103, 0.15751665, 0.15697261]),
 array([0.23894536, 0.20022631, 0.17945785, 0.16924586, 0.16387519,
        0.16077428, 0.15896039, 0.15786233, 0.15719795, 0.15680156]))

In [12]:
trainer = AutoencoderFlowTrainer(ae, output_filename="../figures/training/linear/linear_phase2")
trainer.train(
    dataset=mnist,
    loss_functions=[nll],
    loss_labels=["NLL"],
    batch_size=256,
    epochs=10,
    verbose="all",
    initial_lr=1.e-3,
    final_lr=1.e-5,
    parameters=ae.flow.parameters()
)

NameError: name 'AutoencoderFlowTrainer' is not defined

## Visualize latent space

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h, u = ae.latent(x)
x_out = ae.decoder(h)
h = h.detach().numpy().reshape((-1,2))
u = u.detach().numpy().reshape((-1,2))

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
for i in range(10):
    plt.scatter(h[y==i][:,0], h[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
    
ax = plt.subplot(1,2,2)
for i in range(10):
    plt.scatter(u[y==i][:,0], u[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()

plt.tight_layout()
plt.show()


## Visualize reconstruction

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h = ae.encoder(x)
x_out = ae.decoder(h)
h = h.detach().numpy()
x_out = x_out.detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(18):
    ax = plt.subplot(6, 6, 2*i + 1)
    plt.imshow(x[i].reshape((28,28)))
    ax = plt.subplot(6, 6, 2*i + 2)
    plt.imshow(x_out[i].reshape((28,28)))
        
plt.tight_layout()
plt.show()